### 3-dimensionale Datenflüsse

Das erste Kennzeichen von Flow-Design Datenflüssen ist die Abwesenheit von Schleifen. Daten fließen nur downstream: "Vorwärts immer, rückwärts nimmer!"😉 ist das Motto.

Der Verzicht auf Schleifen ist eine "freiwillige Selbstbeschränkung". Er trägt dazu bei, dass Datenflüsse deklarativ sind, und erhöht die Verständlichkeit. Schleifen fordern mehr kognitiven Aufwand bei der Analyse von Code, weil sie dazu zwingen, etwas im Gedächtnis zu behalten, z.B. den Zustand einer Schleifenvariablen. Außerdem vergrößern sie das Risiko eines Bug z.B. durch 1-off Fehler:

```
var numbers = new[]{1,2,3};
var sum = 0;
for(var i=0; i<=numbers.Length; i++) //😱
    sum += numbers[i];
```

Moderne, abstraktere Sprachkonstrukte verringern zwar solches Risiko...

```
var numbers = new[]{1,2,3};
var sum = 0;
foreach(var n in numbers) //😇
    sum += numbers[i];
```

...und\/oder erhöhen die Verständlichkeit:

```
var numbers = new[]{1,2,3};
var sum = numbers.Sum(); //🥳
```

Letztlich bleiben Schleifen aber problematisch: Es sind Kontrollstrukturen, die zur Schachtelung einladen und unübersichtlich werden, wenn sie wachsen. Nicht in allen Fällen helfen moderne Abstraktionen. Deshalb gilt das Prinzip des Verzichts auf Schleifen in Datenflüssen.

Doch wie sieht es mit Fallunterscheidungen aus? Lassen sie sich auch vermeiden? Nein, Fallunterscheidungen definieren alternative Verhalten. Das ist notwendig und nicht zu vermeiden, auch wenn die Verhaltensproduktion mit Datenflüssen modelliert wird.

#### Kategorisierung

Eine häufige Anforderung im Rahmen der Verhaltensproduktion ist die Kategorisierung von Daten mit anschließend unterschiedlicher Behandlung je nach Kategorie. Beispiel: Eine Eingabe soll in eine Dezimalzahl konvertiert werden, wenn sie eine binäre Zahl ist, oder umgekehrt. Eine binäre Zahl liegt vor, wenn die Eingabe auf "b" endet.

Eine Lösung könnte mit Flow-Design so aussehen:

![](images/df109.png)

Neu ist hier, dass eine Funktionseinheit wie `Determine number system` mehrere Ausgänge haben kann. Die stehen für die Alternativen, dass der Input als binäre oder dezimale Zahl vorliegt. Jenachdem, zu welcher Kategorie der Input gehört, fließt die Verarbeitung entlang des einen oder des anderen Datenflussarms weiter.

In [129]:
class NumberConverter {
    public static string Convert(string number) {
        var result = "";
        DetermineNumberSystem(number,
            decimalNumber => result = ToBinary(decimalNumber),
            binaryNumber => result = ToDecimal(binaryNumber)
        );
        return result;
    }
    
    private static void DetermineNumberSystem(string number, Action<string> onIsDecimal, Action<string> onIsBinary) {
        if (number.EndsWith("b"))
            onIsBinary(number.Substring(0, number.Length-1));
        else
            onIsDecimal(number);
    }
    
    
    private static string ToBinary(string number)
        => System.Convert.ToString(int.Parse(number), 2);
    
    private static string ToDecimal(string number)
        => System.Convert.ToInt32(number,2).ToString();
}


display($"10b={NumberConverter.Convert("10b")}");
display($"10={NumberConverter.Convert("10")}");

10b=2

10=1010

Mehrere parallele Datenflüsse fügen den bisherigen 2-dimensionalen Modellen eine dritte Dimension hinzu:

![](images/df109a.png)

Wie schon vorher bei den 1-dimensionalen Datenflüssen gilt auch hier: alle Funktionseinheiten arbeiten grundsätzlich unabhängig voneinander. Nacheinander geschaltete Funktionseinheiten können gleichzeitig aktiv sein, parallel geschaltete ebenfalls. Dass das oft nicht der Fall ist, weil nicht nötig wie im obigen Beispiel, tut dem keinen Abbruch. 3-dimensionale Datenflüsse erlauben die modellierung synchroner wie asynchroner, gar paralleler Verarbeitung.

##### Übersetzungsalternativen

Auffallend an den alternativen Ausgängen der Funktionseinheit `DetermineNumberSystem` ist, dass Output auf beiden als stream herausfließt. Das ist immer der Fall, wenn Output-Nachrichten entstehen können oder auch nicht. Wenn der Input eine dezimale Zahl darstellt, dann fließt Output auf dem zugehörigen Port heraus - und beim anderen nicht. Und umgekehrt. Nur mit streams ist es möglich, keinen Output zu erzeugen.

Das führt dann dazu, dass in der Übersetzung continuations zum Einsatz kommen. Nur mit ihnen lässt sich der Fall ohne Output für eine Alternative sauber implementieren.

Alternativ wäre in einer Sprache wie C# mit Tupel-Unterstützung zwar auch folgende Übersetzung möglich:

In [131]:
class NumberConverter_v2 {
    public static string Convert(string number) {
        var categorization = DetermineNumberSystem(number);
        if (categorization.isBinary)
            return ToDecimal(categorization.normalizedNumber);
        else
            return ToBinary(categorization.normalizedNumber);
    }
    
    private static (bool isBinary, string normalizedNumber) DetermineNumberSystem(string number) {
        if (number.EndsWith("b"))
            return (true, number.Substring(0, number.Length-1));
        else
            return (false, number);
    }
    
    
    private static string ToBinary(string number)
        => System.Convert.ToString(int.Parse(number), 2);
    
    private static string ToDecimal(string number)
        => System.Convert.ToInt32(number,2).ToString();
}


display($"10b={NumberConverter_v2.Convert("10b")}");
display($"10={NumberConverter_v2.Convert("10")}");

10b=2

10=1010

Doch damit würde die Integration `Convert` durch eine Kontrollstruktur verschmutzt. Das ist zwar nur minimal... dennoch sollte man sich dessen bewusst sein.

Im allgemeinen Fall würde diese Variante die Kategorie über einen `enum`-Datentyp zurückliefern.

Oder sie könnte sogar noch weitergehen und die downstream Verarbeitung selbst dynamisch bestimmen. Hierin könnte man zwar einen Widerspruch zum PoMO sehen... doch wenn Kategorisierung und Mapping auf eine Verarbeitung getrennt sind, mag das der Verständlichkeit und Testbarkeit nicht abträglich, sondern sogar zuträglich sein.

![](images/df110.png)

Jetzt gibt es keine Continuations mehr im Code. Jetzt gibt es keine unsauberen Kontrollstrukturen mehr in Integrationen.

In [136]:
class NumberConverter_v3 {
    public static string Convert(string number) {
        var conversion = ChooseConverter(number);
        return conversion.convert(conversion.normalizedNumber);
    }
    
    private static (Func<string,string> convert, string normalizedNumber) ChooseConverter(string number) {
        var categorization = DetermineNumberSystem(number);
        var convert = PickConverter(categorization.isBinary);
        return (convert, categorization.normalizedNumber);
    }
    
    private static (bool isBinary, string normalizedNumber) DetermineNumberSystem(string number) {
        if (number.EndsWith("b"))
            return (true, number.Substring(0, number.Length-1));
        else
            return (false, number);
    }
    
    private static Func<string,string> PickConverter(bool isBinary) => isBinary switch {
        true => ToDecimal,
        false => ToBinary
    };
    
    
    private static string ToBinary(string number)
        => System.Convert.ToString(int.Parse(number), 2);
    
    private static string ToDecimal(string number)
        => System.Convert.ToInt32(number,2).ToString();
}


display($"10b={NumberConverter_v3.Convert("10b")}");
display($"10={NumberConverter_v3.Convert("10")}");

10b=2

10=1010

Dem Anfänger in Sachen Flow-Design fällt es gewöhnlich nicht leicht, sich vorzustellen, dass Kontrollstrukturen und insbesondere Fallunterscheidungen wirklich nur in den Blättern des Zerlegungsbaumes stehen. Das ist verständlich, weil es so ungewohnt ist. In den Bäumen tiefer funktionaler Abhängigkeiten ist das undenkbar und scheint auch gar nicht nötig.

Doch gerade hierin liegt das Geheimnis der Verständlichkeit und Testbarkeit von Code, der mit Flow-Design modelliert und IOSP/PoMO folgend übersetzt wird! Deshalb sollten alle Anstrengungen unternommen werden, Schleifen und Fallunterscheidungen wirklich in die Operation-Funktionen hinunter zu drücken. Es ist in jedem Fall machbar und ist der Default im Flow-Design. Abweichungen davon sollten sehr bewusst eingegangen werden; sie lassen dark logic entstehen und verwässern das stratified design.

#### Fehler

Ein weiterer typischer Fall für parallele Datenflüsse stellen Fehler dar. Da gibt es einerseits den Datenfluss für den "happy day", d.h. wenn alles wie erwartet läuft. Und andererseits gibt es den Datenfluss für den "rainy day", d.h. den Fehlerfall.

Ein typisches Szenario ist die Validation: Wenn ankommende Daten bestimmten Kriterien genügen, scheint die Sonne, ansonsten regnet es. Gutfall und Fehlerfall zu unterscheiden ist ein nicht zu vernachlässigender Teil jedes Modells. Im Flow-Design geschieht das wieder mit mehreren Ausgängen und parallelen Datenflüssen.

Als Beispiel die Addition mehrerer durch Leerzeichen getrennter Zahlen. Sind alle Zahlen valide, kann die Summe gebildet und ausgegeben werden. Falls nicht, gibt es eine Fehlermeldung.

![](images/df111.png)

Gutfall oder Fehlerfall sind also im Grunde nur zwei Kategorien. Deren Bestimmung erfolgt oft jedoch nicht in einer speziellen Funktionseinheit zur Kategorisierung wie bei der Zahlenwandlung oben, sondern ist ein Beiprodukt. Hier nimmt z.B. `Parse` den Gutfall an, ist jedoch darauf vorbereitet, dass es zum Fehlerfall kommt.

In [142]:
public class StringMath {
    public static void Sum(string source, Action<int> onOk, Action<string> onError) {
        Parse(source,
             numbers => {
                 var sum = numbers.Sum();
                 onOk(sum);
             },
             erroneousNumber => {
                 var error = $"Cannot sum numbers! '{erroneousNumber}' is not a number.";
                 onError(error);
             }
         );
    }
    
    private static void Parse(string source, Action<int[]> onSuccess, Action<string> onFailure) {
        var candidateNumbers = source.Split(' ');
        var numbers = new List<int>();
        foreach(var cn in candidateNumbers)
            if (int.TryParse(cn, out var n))
                numbers.Add(n);
            else {
                onFailure(cn);
                return;
            }
        onSuccess(numbers.ToArray());
    }
}


var input = "1 2 3";
StringMath.Sum(input,
    sum => display($"sum of {input} = {sum}"),
    error => display($"Error: {error}")
);

input = "1 foo 3";
StringMath.Sum(input,
    sum => display($"sum of {input} = {sum}"),
    error => display($"Error: {error}")
);

sum of 1 2 3 = 6

Error: Cannot sum numbers! 'foo' is not a number.

Das der Gutfall der wünschenswerte und meist interessantere und hoffentlich auch der häufigere ist, könnte man versucht sein, die Asymmetrie zwischen Gutfall und Fehlerfall im Code auszudrücken. Warum nicht den Output des Gutfalls über das Funktionsresultat zurückliefern und den Fehlerfall in einer continuation abhandeln?

```csharp
private static int[] Parse(string source, Action<string> onFailure) {
    var candidateNumbers = source.Split(' ');
    var numbers = new List<int>();
    foreach(var cn in candidateNumbers)
        if (int.TryParse(cn, out var n))
            numbers.Add(n);
        else {
            onFailure(cn);
            return null; //😱
        }
    return numbers.ToArray();
}
```

Das Problem ist der Rückgabewert der Funktion im Fehlerfall. Warum sollte der wie hier `null` sein? Darauf muss die Gutfall-Bearbeitung vorbereitet sein. Das vergrößert die Komplexität des Codes.

Einer symmetrische Behandlung ist der Vorzug zu geben. Wenn die nicht durch continuations stattfinden soll, dann wäre es besser, die Alternative Klassifizierungsübersetzung zum Einsatz zu bringen:

```csharp
private static (bool success, int[] numbers, string error) Parse(string source) {
    var candidateNumbers = source.Split(' ');
    var numbers = new List<int>();
    foreach(var cn in candidateNumbers)
        if (int.TryParse(cn, out var n))
            numbers.Add(n);
        else 
            return (false, new int[0], cn);
    return (true, numbers.ToArray(), "");
}
```

Das ist ein anerkanntes Muster neben der einfacheren Variante einer `Try`-Funktion, z.B. `bool TryParse(string source, out int[] numbers)`.

Ohne continuations rutscht aber in jedem Fall Kontrollstruktur-Logik in die Integration, in der eine solche Funktion aufgerufen wird. Es ist also sensibel abzuwägen.

#### Ausnahmen

Es gibt erwartete Fehler und unerwartet. Letztere werden im Notfall durch Exceptions angezeigt. Die können wie rainy-day Fehler modelliert werden und insofern erwartet aussehen. Oder man benutzt eine spezielle Datenflussnotation, um anzuzeigen, dass Ausnahme nicht näher bestimmt aus einer Reihe von Funktionseinheiten herausspringen können.

Als Beispiel ein Datenfluss der eine Reihe von Dateien verarbeitet: Sie werden gelesen, analysiert und das Analyseergebnis am Ende ausgegeben. In jedem der Schritte könnte eine Ausnahme geworfen werden. Das wird nicht erwartet, aber die Möglichkeit besteht. Deshalb gibt es keine speziellen Fehlerausgänge bei allen Funktionseinheiten, sondern sie werden in einen "Kontext" gesteckt, der allgemein darauf reagiert. (Die Funktionale Programmierung könnte dazu wohl auch *Monade* sagen.)

![](images/df112.png)

Für den möglichen, aber nicht wahrscheinlichen Fall einer Ausnahme jede Funktionseinheit mit einem speziellen Output-Port dafür zu versehen, würde die Komplexität des Modells und des Codes stark erhöhen.

In [152]:
class FileStats {
    public static void CountLines(string path) {
        try {
            (int numberOfFiles, int totalNumberOfLines) accumulator = (0,0);
            EnumerateFiles(path,
                filename => {
                    var lines = System.IO.File.ReadAllLines(filename);
                    accumulator.numberOfFiles += 1;
                    accumulator.totalNumberOfLines += lines.Length;
                });
            DisplayResult(accumulator);
        }
        catch(Exception ex) {
            DisplayException(ex);
        }
    }
    
    
    private static void EnumerateFiles(string path, Action<string> onFile) {
        foreach(var filename in System.IO.Directory.GetFiles(path, "*.*", System.IO.SearchOption.AllDirectories))
            onFile(filename);
    }
    
    
    private static void DisplayResult((int numberOfFiles, int totalNumberOfLines) result)
        => Console.WriteLine($"{result.numberOfFiles} files with {result.totalNumberOfLines} lines in total");
    
    private static void DisplayException(Exception ex)
        => Console.WriteLine($"Exception: {ex.Message}");
}


FileStats.CountLines("../entwurf");
FileStats.CountLines("../foo");

23 files with 28384 lines in total
Exception: Could not find a part of the path '/Users/ralfw/Projects/jupyter-notebooks.github/Notebooks/flow-design-tutorial/foo'.


Die Übersetzung des Kontextes mit `try`-`catch` ist naheliegend. Dass diese Kontrollstruktur jedoch in der Integration liegt, ist ein Widerspruch zum IOSP. Da Verständlichkeit und Testbarkeit dadurch jedoch nicht beeinträchtigt werden, ist das ausnahmsweise jedoch kein Problem.

Wer jedoch auf Nummer sicher gehen will, insbesondere wenn noch weitere Logik in beiden Fällen hinzukommen soll, der lagert die Kontrollstruktur aus in eine eigene Methode wie die folgende:

In [154]:
class Exceptionhandling {
    public static void Try(Action tryThis, Action<Exception> onException) {
        try {
            display($"trying...");
            tryThis();
            display($"succeeded!");
        }
        catch(Exception ex) {
            display($"failed!");
            onException(ex);
        }
    }
}

Zusätzliche Logik ist hier angedeutet durch die Protokollnachrichten. Sie würden die eigentliche Logik verrauschen und sind ein einem "Kontext" als separater Aspekt gut ausgelagert.

Die Lösung sieht dann nur wenig anders aus, enthält aber keine Logik mehr in der Integration `CountLines`:

In [155]:
class FileStats_v2 {
    public static void CountLines(string path) {
        Exceptionhandling.Try(
            () => Analyze(path),
            DisplayException
        );
    }
    
    private static void Analyze(string path) {
        (int numberOfFiles, int totalNumberOfLines) accumulator = (0,0);
        EnumerateFiles(path,
            filename => {
                var lines = System.IO.File.ReadAllLines(filename);
                accumulator.numberOfFiles += 1;
                accumulator.totalNumberOfLines += lines.Length;
            });
        DisplayResult(accumulator);
    }
    
    private static void EnumerateFiles(string path, Action<string> onFile) {
        foreach(var filename in System.IO.Directory.GetFiles(path, "*.*", System.IO.SearchOption.AllDirectories))
            onFile(filename);
    }
    
    
    private static void DisplayResult((int numberOfFiles, int totalNumberOfLines) result)
        => Console.WriteLine($"{result.numberOfFiles} files with {result.totalNumberOfLines} lines in total");
    
    private static void DisplayException(Exception ex)
        => Console.WriteLine($"Exception: {ex.Message}");
}


FileStats_v2.CountLines("../entwurf");
FileStats_v2.CountLines("../foo");

trying...

23 files with 28936 lines in total


succeeded!

trying...

failed!

Exception: Could not find a part of the path '/Users/ralfw/Projects/jupyter-notebooks.github/Notebooks/flow-design-tutorial/foo'.


#### Flüsse teilen und zusammenführen

Mehrere Datenflüsse können nicht nur alternativ durchflossen werden, sondern auch gleichzeitig. Das ist z.B. der Fall, wenn Input Teile enthält, die unabhägig von einander verarbeitet werden können. Von der Datenflussnotation her könnte solche Verarbeitung zwar mit einem 1-dimensionalen Datenfluss modelliert werden, aber klarer ist es, wenn Parallelität auch visuell ausgedrückt wird.

##### Split/Join

Als Beispiel mag die Analyse von Dateien in Bezug auf zwei Aspekte dienen: einerseits soll ihre Zeilenzahl bestimmt werden, andererseits ihre Wortanzahl. Beide Informationen werden anschließend zu einem Ergebnis zusammengeschnürt.

![](images/df113.png)

Nach dem Laden der Datei wird der Datenfluss geteilt (split), beide Verarbeitungszweige laufen unabhängig, am Ende werden deren Teilergebnisse zu einem Gesamtergebnis zusammengeführt (join).

Sofern eine Zusammenführung unaufwändig ist, muss keine eigens benannte Funktionseinheit dafür modelliert werden. Es reicht das join-Symbol wie hier gezeigt.

In [164]:
class FileAnalysis {
    public class Result {
        internal Result(int numberOfLines, int numberOfWords) {
            NumberOfLines = numberOfLines;
            NumberOfWords = numberOfWords;
        }
        
        public int NumberOfLines {get;}
        public int NumberOfWords {get;}
    }
    
    
    public static Result Analyze(string filename) {
        var text = Load(filename);
        return new Result(
            CountLines(text),
            CountWords(text)
        );
    }
    
    
    private static string Load(string filename)
        => System.IO.File.ReadAllText(filename);
    
    
    private static int CountLines(string text) {
        var lines = text.Split('\n');
        return lines.Length;
    }
    
    private static int CountWords(string text) {
        var words = text.Split(new[]{' ', '\t', '\n', '\r'}, StringSplitOptions.RemoveEmptyEntries);
        return words.Count();
    }
}


display(FileAnalysis.Analyze("samples/poem.txt"));

NumberOfLines,NumberOfWords
15,79


Was im Modell noch explizit ist, geschieht im Code dann fast unmerklich:

* Der split findet keine spezielle Übersetzung; dieselben Daten werden einfach als aktuelle Parameter zweier Funktionsaufrufe benutzt.
* Die Parallelität der Datenflüsse ist nicht deutlich zu sehen, weil der lineare Quelltext das nicht erlaubt. Nur aus der Benutzung derselben Daten als Input für aufeinander folgende Funktionsaufrufe ist Parallelität herauszulesen.
* Der join besteht schlicht in der Nutzung beider Teilergebnisse als Parameter eines weiteren Funktionsaufrufs.

In anderen Fällen mögen split und join hingegen sowohl im Modell wie im Code deutlicher zu sehen sein.

![](images/df114.png)

Zweierlei ist dabei dann zu bedenken:

* Split: Fließt Output aus den Ports optional/alternativ oder stets? Optionalität fordert die Modellierung als stream und legt eine Implementation als continuation nahe.
* Join: Soll Verarbeitung bei jedem Eintreffen von Input stattfinden, egal auf welchem Port er hineinfließt, oder nur, wenn an allen Ports Input anliegt? Letzteres lässt sich leicht in mehrere Funktionsparameter übersetzen wie oben beim `Result`-Konstruktor zu sehen. Ersteres legt eine Übersetzung in mehrere separate Funktionen einer Klasse nahe.

##### Scatter/Gather

Ein Beispiel für explizite splits und joins sind echte Parallelverarbeitungsszenarien, d.h. Übersetzungen, in denen mehrere Threads zum Einsatz kommen.

Beispiel: Dateien sollen parallel analysiert werden. Anschließend werden alle Analyseergebnisse zu einem zusammengeführt. Die Analyse kann so simpel sein wie die Zählung der Zeilen je Datei.

![](images/df115.png)

Zunächst findet hier eine unbekannt große Aufsplittung des Datenflusses in parallele/konkurrente Flüsse statt (scatter). Das kann mit einem stream geschehen, bei dem für jedes Element ein Thread gestartet wird.

Wie lange die einzelnen Analysen dauern und wieviele es sind, ist bei der Akkumulation des Ergebnisses jedoch nicht bekannt (gather). Wann soll also deren Output ausfließen? Wie weiß die Akkumulation, dass das letzte Analyseergebnis eingetroffen ist? Das kann implizit mittels einer Framework-Abstraktion geschehen (z.B. ein `Task.WaitAll` im .NET Framework) oder explizit durch Vergleich der Zahl der eingetroffenen Ergebnisse mit der zu erwartenden Zahl, die von scatter gemeldet wird. Das Modell zeigt den zweiten Ansatz für gather, um technologieneutraler und expliziter zu sein.

Modelle, in denen Parallelverarbeitung eine Rolle spielt, können durch Farben deutlich machen, wie Funktionseinheiten auf Threads verteilt sind. Dadurch kann z.B. Synchronisationsbedarf erkannt und modelliert werden.

In [12]:
using System.Threading;


class ConcurrentFileAnalysis {
    public static (int numberOfFiles, int totalNumberOfLines) Analyze(string path) {
        var acc = new Accumulator();
        
        var filenames = CompileFiles(path);
        Scatter(filenames,
            filename => {
                var numberOfLines = AnalyzeFile(filename);
                acc.Add(numberOfLines);
            },
            acc.SetNumberOfValuesToExpect
        );
            
        return acc.Gather();
    }
    
    private static string[] CompileFiles(string path) {
        var filenames = System.IO.Directory.GetFiles(path, "*.ipynb", System.IO.SearchOption.AllDirectories);
        return filenames.Where(filename => filename.IndexOf("/.") < 0).ToArray();
    }
    
    private static void Scatter(string[] filenames, Action<string> onFilename, Action<int> onAllScattered) {
        foreach(var filename in filenames) {
            var th = new Thread(() => onFilename(filename));
            th.Start();
        }
        onAllScattered(filenames.Length);
    }
    
    private static int AnalyzeFile(string filename) {
        display($"  analyzing {filename}");
        var lines = System.IO.File.ReadAllLines(filename);
        return lines.Length;
    }
}


private class Accumulator {
    private object _lock = new Object();
    private AutoResetEvent _are = new AutoResetEvent(false);

    private int _numberOfValuesToExpect = 0;
    private int _numberOfValuesReceived = 0;

    private int _total;

    
    public void SetNumberOfValuesToExpect(int numberOfValuesToExpect) {
        lock(_lock) {
            _numberOfValuesToExpect = numberOfValuesToExpect;
            
            if (_numberOfValuesReceived >= _numberOfValuesToExpect)
                _are.Set();
        }
    }

    public void Add(int value) {
        lock(_lock) {
            _numberOfValuesReceived++;
             _total += value;
            
            if (_numberOfValuesToExpect > 0 && _numberOfValuesReceived >= _numberOfValuesToExpect)
                 _are.Set();
        }
    }
    
    public (int numberOfValues, int totalValue) Gather() {
        _are.WaitOne();
        return (_numberOfValuesToExpect, _total);
    }
}


var result = ConcurrentFileAnalysis.Analyze("..");
display($"{result.numberOfFiles} files with {result.totalNumberOfLines} lines total");

  analyzing ../prozess.ipynb

  analyzing ../anforderungen.ipynb

  analyzing ../anforderung-logik-lücke.ipynb

  analyzing ../index.ipynb

  analyzing ../entwurf/entwurf_1.ipynb

...

  analyzing ../analyse/slicing.ipynb

  analyzing ../anatomie/radikale_oo.ipynb

  analyzing ../anatomie/ioda.ipynb

  analyzing ../anatomie/dimensionen.ipynb

10 files with 5601 lines total

Die Implementation des Modells benutzt bewusst Multi-Threading-Primitiven, um für Leser mit unterschiedlichen Plattformerfahrungen verständlich zu sein.

Der `Accumulator` ist als eigene Klasse herausgezogen, um seinen Zustand zu isolieren und mit den davon abhängigen Funktionseinheiten zusammenzufassen. Auf weitere Klassen wurde bewusst verzichtet, um den Fokus auf den Funktionen für die Funktionseinheiten im Modell zu halten. Dadurch ist die Accumulation für sich genommen auch leicht testbar.

Die weiteren Funktionseinheiten sind nicht über Klassen differenziert. Angesichts ihrer sehr unterschiedlichen Verantwortlichkeiten ist das ultimativ nicht wünschenswert, für den Zweck hier jedoch ausreichend. Mehr dazu beim Thema Module. Wesentlich ist, dass die Verantwortlichkeiten überhaupt in eigene Funktionen vorliegen, die grundsätzlich leicht zu testen sind. Eine `private` Sichtbarkeit scheint dem im Wege zu stehen, ist aber nicht wirklich ein Problem. Mehr dazu beim Thema Codierung.

### Zusammenfassung

Datenflüsse sind die intuitive Übersetzung von IOSP und PoMO in eine Modellierungssprache. Sie sind leichtgewichtig als Notation und reich an deklarativer Ausdrucksfähigkeit. Für Flow-Design stellen sie den Startpunkt jeder Modellierung von Softwaresystemen dar, sobald klar ist, welche Nachrichten aus der Umwelt zu verarbeiten sind.

Es braucht eine gewisse Zeit, das gewohnheitsmäßige imperative und detailfokussierte Denken hinter sich zu lassen. Das lässt Datenflussmodellierung am Anfang durchaus umständlich oder nicht ausdrucksstark genug erscheinen. Mit etwas gutem Durchhaltewillen wird dieses Tal der Tränen jedoch zügig durchschritten - um am Ende zurückzuschauen mit einem Kopfschütteln, wie jemals weniger visuell, weniger verständlich und vor allem mit funktionalen Abhängigkeiten modelliert und implementiert werden konnte.

Das bedeutet nicht, dass Datenflüsse der einzige Modellierungspfeil im Köcher sein sollten. Insbesondere Zustandsautomaten bieten sich immer wieder als Alternative an, wenn ein Datenflussmodell umständlich zu werden droht. Dennoch steht Flow-Design für die Überzeugung, dass Datenflüsse der Einstiegspunkt der Nachdenkens über Softwarelösungen sein sollten. Software transformiert Daten; Transformationen laufen in Prozessen ab; Prozesse bestehen aus Schritten, die gegenseitig Produzenten und Konsumenten von Daten sind. Das ist eine natürliche Sichtweise auf die Struktur und Arbeitsweise von Software wie von Organisationen.

Nicht nur braucht es eine gewisse Zeit, überhaupt (mit Datenflüssen) zu modellieren, bevor man codiert. Es braucht auch Gewöhnung an die zeitlich begrenzte Bedeutung dieser Modelle. Deshalb sind die Zeichnungen in diesen Notizbüchern auch alle skizzenhaft angelegt: das unterstreicht ihre temporäre und persönliche Bedeutung.

Datenflüsse können natürlich zur Dokumentation angefertigt und abgeheftet werden. Dazu hat Flow-Design jedoch eher keine Meinung. Flow-Design beschäftigt sich nicht mit Dokumentation von Vorhandenem, sondern mit dem kreativien Prozess des Erfindens von Neuem. Auf diesem Weg will Flow-Design mit Notation, Prinzipien und Praktiken unterstützen. Es besteht deshalb kein Anspruch, dass Modelle über längere Zeit Gültigkeit haben oder von Personen jenseits des Erstellerkreises verstanden werden.

Selbstverständlich soll ihre visuelle Abstraktion helfen, Lösungsansätze im Kopf zu behalten, besser überblicken und diskutieren zu können. Das Wesentliche passiert dabei jedoch weiterhin in den Köpfen der Beteiligten. Die visuellen Modelle dienen dabei zum schnellen Abgleich von mentalen Modellen. Verschwinden die Köpfe, die sie ersonnen haben, verschwindet viel ihres Wertes.

Wer mit einem Flow-Design konfrontiert ist, sollte sich daher nach einem Führer durch die Datenflüsse umsehen. Das ist also nicht nur nötig beim einer ersten Konfrontation mit Code, sondern auch bei Modellen. Ihre Visualität darf nicht darüber hinwegtäuschen, dass auch sie nur unvollständige Wiedergaben von mentalen Modellen sind. Wer nur ein Modell betrachtet, dem fehlen Informationen. Die durch Deutung zu beschaffen, ist mühsam.

Für Code ist der Intrepreations- und Deutungsaufwand natürlich noch viel größer. Doch auch für Modelle besteht er. Entwurfssitzungen erzeugen also keine zeitlose Dokumentation, sondern nur beim Denken und Codieren unterstützende Skizzen - von denen in der Implementation mit Augenmaß auch abgewichen werden kann.

Die wohlfeile Kritik, dass Entwurfsdiagramme zu schnell inkonsistent mit dem Code werden würden, als dass sie nützlich seien, perlt an Flow-Design ab. Flow-Design hat keinen auch nur mittelfristigen Konsistenzanspruch für seine Modelle. Sie dienen im Moment des Entwurfs zur Unterstützung des eigenen Denkens und der Kommunikation im Team. Nicht mehr und nicht weniger. Anschließend leiten sie die Codierung, aus der mit ihnen einige implizite ad hoc Kreativität herausgepresst wird. Das beschleunigt die Codierung und führt zu Code auf hohem, vergleichbarem Sauberkeitsniveau, der im Review wenige Reibungspunkte bieten sollte.

Nach dem Review haben die Modelle ihre Schuldigkeit getan. Die Wahrheit liegt dann im Code - der sich angesichts der IOSP/PoMO-Konformität jedoch leicht zurückübersetzen lässt in Modelle.